In [13]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import os
from tensorflow import keras
from tensorflow.keras.callbacks import ModelCheckpoint

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [14]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=['EIN', 'ORGANIZATION','SPECIAL_CONSIDERATIONS'])
application_df

,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,STATUS,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,1,0,5000,1
1,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,1,1-9999,108590,1
2,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,1,0,5000,0
3,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,1,10000-24999,6692,1
4,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,1,100000-499999,142590,1
...,...,...,...,...,...,...,...,...,...
34294,THE LIONS CLUB OF HONOLULU KAMEHAMEHA,T4,Independent,C1000,ProductDev,1,0,5000,0
34295,INTERNATIONAL ASSOCIATION OF LIONS CLUBS,T4,CompanySponsored,C3000,ProductDev,1,0,5000,0
34296,PTA HAWAII CONGRESS,T3,CompanySponsored,C2000,Preservation,1,0,5000,0
34297,AMERICAN FEDERATION OF GOVERNMENT EMPLOYEES LO...,T5,Independent,C3000,ProductDev,1,0,5000,1


In [15]:
# Determine the number of unique values in each column.
application_df.nunique()


NAME                19568
APPLICATION_TYPE       17
AFFILIATION             6
CLASSIFICATION         71
USE_CASE                5
STATUS                  2
INCOME_AMT              9
ASK_AMT              8747
IS_SUCCESSFUL           2
dtype: int64

In [16]:
# Look at APPLICATION_TYPE value counts for binning
counts = application_df['APPLICATION_TYPE'].value_counts()
counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [17]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
#  YOUR CODE GOES HERE
application_types_replace = list(counts[counts<500].index)
application_types_replace

['T9', 'T13', 'T12', 'T2', 'T25', 'T14', 'T29', 'T15', 'T17']

In [18]:
# Replace in dataframe
for app in application_types_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [19]:
# Look at CLASSIFICATION value counts for binning
classification = application_df["CLASSIFICATION"].value_counts()
classification

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [20]:
# You may find it helpful to look at CLASSIFICATION value counts >1
counts_classification = classification[classification>1]
counts_classification

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [21]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(classification[classification<1000].index)

In [22]:
# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [23]:
count_names = application_df['NAME'].value_counts()
count_names

PARENT BOOSTER USA INC                                                  1260
TOPS CLUB INC                                                            765
UNITED STATES BOWLING CONGRESS INC                                       700
WASHINGTON STATE UNIVERSITY                                              492
AMATEUR ATHLETIC UNION OF THE UNITED STATES INC                          408
                                                                        ... 
ST LOUIS SLAM WOMENS FOOTBALL                                              1
AIESEC ALUMNI IBEROAMERICA CORP                                            1
WEALLBLEEDRED ORG INC                                                      1
AMERICAN SOCIETY FOR STANDARDS IN MEDIUMSHIP & PSYCHICAL INVESTIGATI       1
WATERHOUSE CHARITABLE TR                                                   1
Name: NAME, Length: 19568, dtype: int64

In [24]:
names_to_replace = list(count_names[count_names<21].index)

for name in names_to_replace:
    application_df["NAME"] = application_df["NAME"].replace(name, "Other")
    
# Check to make sure binning was successful
application_df["NAME"].value_counts()

Other                                              22469
PARENT BOOSTER USA INC                              1260
TOPS CLUB INC                                        765
UNITED STATES BOWLING CONGRESS INC                   700
WASHINGTON STATE UNIVERSITY                          492
                                                   ...  
FULL GOSPEL BUSINESS MENS FELLOWSHIP IN AMERICA       21
ARIZONA FEDERATION OF GARDEN CLUBS                    21
ALPHA PHI SIGMA INC                                   21
AMERICAN IRIS SOCIETY                                 21
FRATERNAL ORDER OF POLICE                             21
Name: NAME, Length: 119, dtype: int64

In [26]:
numeric_df = pd.get_dummies(application_df)
numeric_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,NAME_AIR FORCE ASSOCIATION,NAME_ALABAMA FEDERATION OF WOMENS CLUBS,NAME_ALABAMA TREASURE FOREST ASSOCIATION,NAME_ALPHA PHI SIGMA,NAME_ALPHA PHI SIGMA INC,NAME_ALTRUSA INTERNATIONAL FOUNDATION INC,NAME_AMATEUR ATHLETIC UNION OF THE UNITED STATES,...,USE_CASE_ProductDev,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,1,5000,1,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
1,1,108590,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,1,5000,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
3,1,6692,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,1,142590,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [27]:
# Split our preprocessed data into our features and target arrays
X = numeric_df.drop("IS_SUCCESSFUL", axis=1)
y = numeric_df["IS_SUCCESSFUL"]

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [28]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

Compile, Train and Evaluate the Model

In [29]:
# Size of data 
input_features = X.shape[1]

print("Input Features:",input_features)

# Size of the output layer 
output_layer_size = 1

print("Output Layer Size:", output_layer_size)
 
# The number of hidden neurons 
hidden_nodes_1 = (input_features * 2) - 1

print("Hidden nodes in layer 1:", hidden_nodes_1)

# The number of hidden neurons 
nodes_2 = input_features * (2/3)

hidden_nodes_2 = round(nodes_2,0) + output_layer_size

print("Hidden nodes in layer 2:", int(hidden_nodes_2))

Input Features: 156
Output Layer Size: 1
Hidden nodes in layer 1: 311
Hidden nodes in layer 2: 105


In [30]:
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_1, activation="tanh", input_dim=input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_2, activation="tanh"))

# Output layer
nn.add(tf.keras.layers.Dense(units=output_layer_size, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 311)               48827     
                                                                 
 dense_1 (Dense)             (None, 105)               32760     
                                                                 
 dense_2 (Dense)             (None, 1)                 106       
                                                                 
Total params: 81,693
Trainable params: 81,693
Non-trainable params: 0
_________________________________________________________________


In [31]:
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [33]:
os.makedirs('checkpoints/', exist_ok=True)
checkpoint_paths = 'checkpoints/weights.{epoch}.hdf5'

cp_callback = ModelCheckpoint(filepath=checkpoint_paths,
                              verbose=1,
                              save_weights_only=True,
                              save_freq='epoch',
                              period=5)

In [34]:
# Train the model
model_fitting = nn.fit(X_train_scaled, y_train, epochs=100, callbacks = [cp_callback])

Epoch 1/100
804/804 [==============================] - 4s 4ms/step - loss: 0.4899 - accuracy: 0.7553
Epoch 2/100
804/804 [==============================] - 2s 3ms/step - loss: 0.4690 - accuracy: 0.7694
Epoch 3/100
804/804 [==============================] - 3s 4ms/step - loss: 0.4626 - accuracy: 0.7742
Epoch 4/100
804/804 [==============================] - 2s 3ms/step - loss: 0.4613 - accuracy: 0.7748
Epoch 5/100
795/804 [============================>.] - ETA: 0s - loss: 0.4595 - accuracy: 0.7762
Epoch 5: saving model to checkpoints/weights.5.hdf5
804/804 [==============================] - 3s 4ms/step - loss: 0.4591 - accuracy: 0.7764
Epoch 6/100
804/804 [==============================] - 3s 3ms/step - loss: 0.4579 - accuracy: 0.7759
Epoch 7/100
804/804 [==============================] - 2s 3ms/step - loss: 0.4561 - accuracy: 0.7778
Epoch 8/100
804/804 [==============================] - 2s 3ms/step - loss: 0.4551 - accuracy: 0.7782
Epoch 9/100
804/804 [==============================] - 

In [35]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.4802 - accuracy: 0.7684 - 591ms/epoch - 2ms/step
Loss: 0.4801827371120453, Accuracy: 0.7683964967727661


In [36]:
# Export our model to HDF5 file
nn.save('ASC_4.h5')